## Defines

In [12]:
BRAND = "BMW"
IMG_SIZE = (256, 256)
BATCH_SIZE = 32
TOTAL_POSITIVE = 1000
TOTAL_NEGATIVE = 1000
SPLIT_RATIOS = {'train': 0.7, 'val': 0.15, 'test': 0.15}

logo_dir = '/kaggle/input/logo-dataset-2341-classes-and-167140-images/datasetcopy/trainandtest/train/Transportation/BMW'
background_dir = '/kaggle/input/stanford-background-dataset/images'
dataset_dir = '/kaggle/working/dataset'
video_path = '/kaggle/input/lab6video/StockFootage.mp4'
save_model_dir = '/kaggle/working/model'

output_video = '/kaggle/working/output_with_boxes.mp4'

print('Logo dir:', logo_dir)
print('Background dir:', background_dir)
print('Dataset dir:', dataset_dir)
print('Video path:', video_path)
print('Save model dir:', save_model_dir)


Logo dir: /kaggle/input/logo-dataset-2341-classes-and-167140-images/datasetcopy/trainandtest/train/Transportation/BMW
Background dir: /kaggle/input/stanford-background-dataset/images
Dataset dir: /kaggle/working/dataset
Video path: /kaggle/input/lab6video/StockFootage.mp4
Save model dir: /kaggle/working/model


## Imports

In [13]:
!pip -q install ultralytics

import albumentations 
import ultralytics, cv2, sys
print('ultralytics', ultralytics.__version__)
print('albumentations', albumentations.__version__)
print('opencv', cv2.__version__)
print('python', sys.version.split()[0])

import os
os.environ["WANDB_MODE"] = "disabled"

ultralytics 8.0.145
albumentations 1.1.0
opencv 4.12.0
python 3.7.12


## Dataset structure

In [14]:
import os, shutil, glob
os.makedirs(dataset_dir, exist_ok=True)
images_dir = os.path.join(dataset_dir, 'images')
labels_dir = os.path.join(dataset_dir, 'labels')
for split in ['train','val','test']:
    os.makedirs(os.path.join(images_dir, split), exist_ok=True)
    os.makedirs(os.path.join(labels_dir, split), exist_ok=True)

print('Created dataset structure under', dataset_dir)


Created dataset structure under /kaggle/working/dataset


## Dataset functions

In [15]:
import random, glob
from PIL import Image
import numpy as np
import os
from tqdm import tqdm

def load_random_logo():
    files = glob.glob(os.path.join(logo_dir, '*'))
    if not files:
        raise FileNotFoundError('No logos found in ' + logo_dir)
    return random.choice(files)

def load_random_bg():
    files = glob.glob(os.path.join(background_dir, '*'))
    if not files:
        raise FileNotFoundError('No background images found in ' + background_dir)
    return random.choice(files)

def place_logo_on_bg(logo_path, bg_path, out_size=IMG_SIZE, scale_range=(0.1,0.4), jitter=0.2):
    bg = Image.open(bg_path).convert('RGBA')
    logo = Image.open(logo_path).convert('RGBA')
    bg = bg.resize(out_size, Image.LANCZOS)
    scale = random.uniform(*scale_range)
    logo_w = int(out_size[0] * scale)
    w_ratio = logo_w / logo.width
    logo_h = max(1, int(logo.height * w_ratio))
    logo = logo.resize((logo_w, logo_h), Image.LANCZOS)
    center_x = out_size[0] // 2 + int(random.uniform(-jitter, jitter) * out_size[0])
    center_y = out_size[1] // 2 + int(random.uniform(-jitter, jitter) * out_size[1])
    xmin = max(0, center_x - logo_w//2)
    ymin = max(0, center_y - logo_h//2)
    xmax = min(out_size[0], xmin + logo_w)
    ymax = min(out_size[1], ymin + logo_h)
    composed = Image.new('RGBA', out_size)
    composed.paste(bg, (0,0))
    composed.alpha_composite(logo, (xmin, ymin))
    return composed.convert('RGB'), (xmin, ymin, xmax, ymax)

def xyxy_to_yolo_norm(bbox, img_w, img_h):
    xmin, ymin, xmax, ymax = bbox
    x_center = (xmin + xmax) / 2.0 / img_w
    y_center = (ymin + ymax) / 2.0 / img_h
    w = (xmax - xmin) / img_w
    h = (ymax - ymin) / img_h
    return x_center, y_center, w, h


## Base dataset generation

In [16]:
import math, uuid

def generate_synthetic_dataset(n_pos=TOTAL_POSITIVE, n_neg=TOTAL_NEGATIVE, out_images_dir=images_dir, out_labels_dir=labels_dir):
    random.seed(42)
    for i in tqdm(range(n_pos), desc='Generating positives'):
        logo_path = load_random_logo()
        bg_path = load_random_bg()
        img, bbox = place_logo_on_bg(logo_path, bg_path)
        fname = f'pos_{i:05d}.jpg'
        r = random.random()
        if r < SPLIT_RATIOS['train']:
            split = 'train'
        elif r < SPLIT_RATIOS['train'] + SPLIT_RATIOS['val']:
            split = 'val'
        else:
            split = 'test'
        img_path = os.path.join(out_images_dir, split, fname)
        os.makedirs(os.path.dirname(img_path), exist_ok=True)
        img.save(img_path, quality=95)
        lbl_path = os.path.join(out_labels_dir, split, os.path.splitext(fname)[0] + '.txt')
        x_center, y_center, w, h = xyxy_to_yolo_norm(bbox, IMG_SIZE[0], IMG_SIZE[1])
        with open(lbl_path, 'w') as f:
            f.write(f"0 {x_center:.6f} {y_center:.6f} {w:.6f} {h:.6f}\n")
    for i in tqdm(range(n_neg), desc='Generating negatives'):
        bg_path = load_random_bg()
        bg = Image.open(bg_path).convert('RGB').resize(IMG_SIZE, Image.LANCZOS)
        fname = f'neg_{i:05d}.jpg'
        r = random.random()
        if r < SPLIT_RATIOS['train']:
            split = 'train'
        elif r < SPLIT_RATIOS['train'] + SPLIT_RATIOS['val']:
            split = 'val'
        else:
            split = 'test'
        img_path = os.path.join(out_images_dir, split, fname)
        os.makedirs(os.path.dirname(img_path), exist_ok=True)
        bg.save(img_path, quality=95)
        lbl_path = os.path.join(out_labels_dir, split, os.path.splitext(fname)[0] + '.txt')
        open(lbl_path, 'w').close()

generate_synthetic_dataset(n_pos=TOTAL_POSITIVE, n_neg=TOTAL_NEGATIVE)
print('Synthetic dataset created.')


Generating negatives: 100%|██████████| 1000/1000 [00:10<00:00, 99.64it/s]

Synthetic dataset created.


## Dataset augmentation via albumentations

In [17]:
from albumentations import (Compose, HorizontalFlip, VerticalFlip, RandomRotate90, ShiftScaleRotate,
                            RandomBrightnessContrast, Blur, GaussNoise, CLAHE, HueSaturationValue,
                            Cutout, RandomShadow)
from albumentations import BboxParams
import cv2

transform = Compose([
    HorizontalFlip(p=0.5),
    VerticalFlip(p=0.1),
    RandomRotate90(p=0.3),
    ShiftScaleRotate(shift_limit=0.0625, scale_limit=0.2, rotate_limit=20, p=0.5, border_mode=0),
    RandomBrightnessContrast(p=0.5),
    HueSaturationValue(p=0.4),
    Cutout(num_holes=8, max_h_size=32, max_w_size=32, p=0.3),
    Blur(blur_limit=3, p=0.2),
    GaussNoise(var_limit=(10.0, 50.0), p=0.2),
    RandomShadow(p=0.2)
], bbox_params=BboxParams(format='yolo', label_fields=['category_ids']))

def augment_and_save_all(n_augs=2):
    for split in ['train','val']:
        img_files = glob.glob(os.path.join(images_dir, split, '*.jpg'))
        for img_path in tqdm(img_files, desc=f'Augment {split}'):
            base = os.path.splitext(os.path.basename(img_path))[0]
            lbl_path = os.path.join(labels_dir, split, base + '.txt')
            boxes = []
            cats = []
            if os.path.exists(lbl_path):
                with open(lbl_path, 'r') as f:
                    for line in f:
                        parts = line.strip().split()
                        if len(parts) != 5:
                            continue
                        cls = int(parts[0])
                        xc, yc, bw, bh = map(float, parts[1:5])
                        boxes.append([xc, yc, bw, bh])
                        cats.append(cls)
            img = cv2.imread(img_path)
            h,w = img.shape[:2]
            for i in range(n_augs):
                try:
                    augmented = transform(image=img, bboxes=boxes, category_ids=cats)
                    aug_img = augmented['image']
                    aug_boxes = augmented['bboxes']
                    aug_cats = augmented['category_ids']
                    if len(aug_boxes) == 0 and len(boxes)>0:
                        continue
                    out_base = f"{base}_aug{i}"
                    out_img_path = os.path.join(images_dir, split, out_base + '.jpg')
                    out_lbl_path = os.path.join(labels_dir, split, out_base + '.txt')
                    cv2.imwrite(out_img_path, aug_img)
                    with open(out_lbl_path, 'w') as f:
                        for cls, b in zip(aug_cats, aug_boxes):
                            f.write(f"{cls} {b[0]:.6f} {b[1]:.6f} {b[2]:.6f} {b[3]:.6f}\n")
                except:
                    pass
    print('Augmentation finished.')

augment_and_save_all()

/opt/conda/lib/python3.7/site-packages/albumentations/augmentations/transforms.py:691: FutureWarning: This class has been deprecated. Please use CoarseDropout
  FutureWarning,
Augment val: 100%|██████████| 981/981 [00:07<00:00, 125.60it/s]

Augmentation finished.


## data.yaml for Ultralytics

In [18]:
data_yaml = {
    'train': os.path.join(dataset_dir, 'images', 'train'),
    'val': os.path.join(dataset_dir, 'images', 'val'),
    'nc': 1,
    'names': ['logo']
}
import yaml
with open(os.path.join(dataset_dir, 'data.yaml'), 'w') as f:
    yaml.dump(data_yaml, f)
print('Wrote', os.path.join(dataset_dir, 'data.yaml'))


Wrote /kaggle/working/dataset/data.yaml


## Model training

In [19]:
from ultralytics import YOLO
os.makedirs(save_model_dir, exist_ok=True)
model = YOLO("/kaggle/input/yolo11/yolov8n.pt") 
model.train(data=os.path.join(dataset_dir, 'data.yaml'), epochs=6, imgsz=640, batch=BATCH_SIZE//2 if BATCH_SIZE>=8 else 8, name='mkr_logo_exp', project="yolo")
print('Training launched. Check runs folder in', save_model_dir)


New https://pypi.org/project/ultralytics/8.3.233 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.0.145 🚀 Python-3.7.12 torch-1.9.1 CUDA:0 (Tesla P100-PCIE-16GB, 16269MiB)
WARNING ⚠️ Upgrade to torch>=2.0.0 for deterministic training.
engine/trainer: task=detect, mode=train, model=/kaggle/input/yolo11/yolov8n.pt, data=/kaggle/working/dataset/data.yaml, epochs=6, patience=50, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=yolo, name=mkr_logo_exp, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=Tr

Training launched. Check runs folder in /kaggle/working/model


## Video inference

In [20]:
import cv2
from ultralytics import YOLO
trained_weights = None
possible = sorted(glob.glob(os.path.join(save_model_dir, 'runs', 'train', 'mkr_logo_exp', 'weights', '*.pt')))
if possible:
    trained_weights = possible[-1]
else:
    trained_weights = '/kaggle/input/yolo11/yolov8n.pt'

print('Using weights:', trained_weights)
model = YOLO(trained_weights)

cap = cv2.VideoCapture(video_path)
if not cap.isOpened():
    raise FileNotFoundError('Cannot open video: ' + video_path)
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
fps = cap.get(cv2.CAP_PROP_FPS) or 30.0
w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
writer = cv2.VideoWriter(output_video, fourcc, fps, (w,h))

from tqdm import tqdm
frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT) or 0)
for _ in tqdm(range(frame_count), desc='Processing video'):
    ret, frame = cap.read()
    if not ret:
        break
    results = model.predict(frame, imgsz=640, verbose=False)
    try:
        res = results[0]
        boxes = res.boxes
        for box in boxes:
            xyxy = box.xyxy[0].cpu().numpy()
            conf = float(box.conf[0]) if hasattr(box, 'conf') else 0.0
            cls = int(box.cls[0]) if hasattr(box, 'cls') else 0
            label = f"{BRAND}:{conf:.2f}"
            x1,y1,x2,y2 = map(int, xyxy)
            cv2.rectangle(frame, (x1,y1), (x2,y2), (0,255,0), 2)
            cv2.putText(frame, label, (x1, max(15,y1-5)), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0,255,0), 2)
    except Exception:
        pass
    writer.write(frame)

cap.release()
writer.release()
print('Saved output video to', output_video)


Using weights: /kaggle/input/yolo11/yolov8n.pt


Processing video: 100%|██████████| 622/622 [00:07<00:00, 82.84it/s]

Saved output video to /kaggle/working/output_with_boxes.mp4
